### Opinion Review 데이터 세트를 이용한 문서 군집화 수행

In [50]:
import pandas as pd
import glob, os

path = r'C:\Users\JAVA01\Python\04_Natural_Language_Processing\05_군집\topics'
all_files = glob.glob(os.path.join(path, '*.data'))

# print(all_files)

In [51]:
filename_list = [] # 파일명들이 저장될 리스트
opinion_text = [] # 파일의 내용(주제)등이 저장될 리스트

1. 개별 파일명들을 filename_list에 저장합니다.
1. 개별파일의 내용은 DataFrame으로 로딩 후 다시 String으로 변환하여 opinion_text 리스트에 저장

In [52]:
# 모든ㄷ 파일네임으로 반복실행
for file_ in all_files :
    # 개별파일을 읽어 DataFrame으로 생성
    df = pd.read_table(file_, index_col=None, header=0, encoding='latin1')
    
    ## 전체 경로에서 파일명만 추출
    filename_ = file_.split('\\')[-1]
    # 추출한 파일명에서 .data를 제외한 나머지 이름 추출
    filename = filename_.split('.')[0]
    # 추출한 파일명은 파일명 리스트에 append
    filename_list.append(filename)
    # 파일의 내용은 opinion_text 리스트에 String 형식으로 변환하여 append
    opinion_text.append(df.to_string())
    
# 취합된 파일명과 파일 내용을 각각에 컬럼으로 담은 프레임 생성
document_df = pd.DataFrame({'filename':filename_list, 'opinion_text':opinion_text})
document_df.head()

,filename,opinion_text
0,accuracy_garmin_nuvi_255W_gps,...
1,bathroom_bestwestern_hotel_sfo,...
2,battery-life_amazon_kindle,...
3,battery-life_ipod_nano_8gb,...
4,battery-life_netbook_1005ha,...


In [53]:
#### Lemmatization을 위한 함수 생성

In [54]:
from nltk.stem import WordNetLemmatizer
import nltk
import string

In [55]:
# 특수기호의 아스키 코드값(key)과 None(value)이 조합된 딕셔너리 생성
remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)
# 어근 추출을 위한 객체 생성
lemmar = WordNetLemmatizer()

In [56]:
# 전달인수로 전달된 text를
# 1. 워드 토큰화(단어단위 분리)
# 2. 소문자로 모두 변환 ->
# 3. remove_punct_dict에 들어있는 key 값을 찾아서 value로 대치
# 4. 결과가 리턴되기전까지 LemTokens함수 호출해서 결과를 받고 리턴함
def LemNormalize(text) :
    return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))

# 위에서 전달된 tokens(분리되고 특수기호 없앤 단어들의 어근 추출을 해서 리턴)
def LemTokens(tokens):
    return [lemmar.lemmatize(token) for token in tokens]

In [57]:
LemNormalize(document_df['opinion_text'][0])

['and',
 'is',
 'very',
 'very',
 'accurate',
 '0',
 'but',
 'for',
 'the',
 'most',
 'part',
 'we',
 'find',
 'that',
 'the',
 'garmin',
 'software',
 'provides',
 'accurate',
 'direction',
 'whereever',
 'we',
 'intend',
 'to',
 'go',
 '1',
 'this',
 'function',
 'is',
 'not',
 'accurate',
 'if',
 'you',
 'dont',
 'leave',
 'it',
 'in',
 'battery',
 'mode',
 'say',
 'when',
 'you',
 'stop',
 'at',
 'the',
 'cracker',
 'barrell',
 'for',
 'lunch',
 'and',
 'to',
 'play',
 'one',
 'of',
 'those',
 'trangle',
 'game',
 'with',
 'the',
 'tee',
 '2',
 'it',
 'provides',
 'immediate',
 'alternative',
 'if',
 'the',
 'route',
 'from',
 'the',
 'online',
 'map',
 'program',
 'wa',
 'inaccurate',
 'or',
 'blocked',
 'by',
 'an',
 'obstacle',
 '3',
 'ive',
 'used',
 'other',
 'gps',
 'unit',
 'a',
 'well',
 'a',
 'gps',
 'built',
 'into',
 'car',
 'and',
 'to',
 'this',
 'day',
 'nothing',
 'beat',
 'the',
 'accuracy',
 'of',
 'a',
 'garmin',
 'gps',
 '4',
 'it',
 'got',
 'me',
 'from',
 'poin

#### TF-IDF 피처 벡터화, TfidfVectorizer에서 피처 벡터화 수행시 Lemmatization을 적용하여 토큰화

In [58]:
from sklearn.feature_extraction.text import TfidfVectorizer
import warnings
warnings.filterwarnings('ignore')

tfidf_vect = TfidfVectorizer(tokenizer=LemNormalize, stop_words='english', ngram_range=(1, 2), min_df=0.05, max_df=0.85)

# opinion_text 컬럼값으로 feature vectorization 수행
feature_vect = tfidf_vect.fit_transform(document_df['opinion_text'])

### 군집 알고리즘(K-Mean)
- 대표적인 비지도 학습 알고리즘입니다.
- 지도학습과 비지도학습의 외형상 차이점은 target이 있느냐 없느냐로 구분합니다.
- train feature 데이터와 train target 데이터를 갖고 학습한 후에, test feature로 테스트하고 결과를 test target과 비교하여 성능평가는 하는 지도학습의 모습과는 달리
- 비지도학습은 target 없이 feature들로만 학습하고 그 결과로 군집된 데이터와 완성된 머신러닝 모델을 얻습니다.

##### 5개의 군집으로 K-Means 군집

In [59]:
from sklearn.cluster import KMeans

In [60]:
# 5개의 집합으로 군집화 객체 생성 및 수행.
# max_iter=10000 군집 알고리즘의 실행횟수 10000회
km_cluster = KMeans(n_clusters=5, max_iter=10000, random_state=0)
km_cluster.fit(feature_vect)

KMeans(max_iter=10000, n_clusters=5, random_state=0)

In [61]:
# 군집 결과 타겟들(0. 1. 2. 3. 4)
cluster_label = km_cluster.labels_
# km_cluster.cluster_centers_ : 군집 중심점과의 거리들
cluster_centers = km_cluster.cluster_centers_
document_df['cluster_label'] = cluster_label
document_df.head()

,filename,opinion_text,cluster_label
0,accuracy_garmin_nuvi_255W_gps,...,2
1,bathroom_bestwestern_hotel_sfo,...,0
2,battery-life_amazon_kindle,...,1
3,battery-life_ipod_nano_8gb,...,1
4,battery-life_netbook_1005ha,...,1


In [62]:
document_df[document_df['cluster_label']==0].sort_values(by='filename')

,filename,opinion_text,cluster_label
1,bathroom_bestwestern_hotel_sfo,...,0
32,room_holiday_inn_london,...,0
30,rooms_bestwestern_hotel_sfo,...,0
31,rooms_swissotel_chicago,...,0


In [63]:
document_df[document_df['cluster_label']==1].sort_values(by='filename')

,filename,opinion_text,cluster_label
2,battery-life_amazon_kindle,...,1
3,battery-life_ipod_nano_8gb,...,1
4,battery-life_netbook_1005ha,...,1
19,keyboard_netbook_1005ha,...,1
26,performance_netbook_1005ha,...,1
41,size_asus_netbook_1005ha,...,1
42,sound_ipod_nano_8gb,headphone jack i got a clear case for it a...,1
44,speed_windows7,...,1


In [64]:
document_df[document_df['cluster_label']==2].sort_values(by='filename')

,filename,opinion_text,cluster_label
0,accuracy_garmin_nuvi_255W_gps,...,2
5,buttons_amazon_kindle,...,2
8,directions_garmin_nuvi_255W_gps,...,2
9,display_garmin_nuvi_255W_gps,...,2
10,eyesight-issues_amazon_kindle,...,2
11,features_windows7,...,2
12,fonts_amazon_kindle,...,2
23,navigation_amazon_kindle,...,2
33,satellite_garmin_nuvi_255W_gps,...,2
34,screen_garmin_nuvi_255W_gps,...,2


In [66]:
document_df[document_df['cluster_label']==3].sort_values(by='filename')

,filename,opinion_text,cluster_label
13,food_holiday_inn_london,...,3
14,food_swissotel_chicago,...,3
15,free_bestwestern_hotel_sfo,...,3
20,location_bestwestern_hotel_sfo,...,3
21,location_holiday_inn_london,...,3
24,parking_bestwestern_hotel_sfo,...,3
27,price_amazon_kindle,...,3
28,price_holiday_inn_london,...,3
38,service_bestwestern_hotel_sfo,...,3
39,service_holiday_inn_london,...,3


In [67]:
# 3개의 집합으로 군집화
km_cluster = KMeans(n_clusters=3, max_iter=10000, random_state=0)
km_cluster.fit(feature_vect)
cluster_label = km_cluster.labels_
# 소속 클러스터를 cluster_label 컬럼으로 할당하고 cluster_label 값으로 정렬
document_df['cluster_label'] = cluster_label

In [68]:
document_df[document_df['cluster_label']==0].sort_values(by='filename')

,filename,opinion_text,cluster_label
0,accuracy_garmin_nuvi_255W_gps,...,0
2,battery-life_amazon_kindle,...,0
3,battery-life_ipod_nano_8gb,...,0
4,battery-life_netbook_1005ha,...,0
5,buttons_amazon_kindle,...,0
8,directions_garmin_nuvi_255W_gps,...,0
9,display_garmin_nuvi_255W_gps,...,0
10,eyesight-issues_amazon_kindle,...,0
11,features_windows7,...,0
12,fonts_amazon_kindle,...,0


- KMeans 객체의 cluster_cneters_ 속성은 개별 피처들의 클러스터 중심과의 상대 위치를 정규화한 값입니다.
- 0~1까지의 값으로 표현되며 1에 가까울수록 중심에 더 가깝다는 의미입니다.

In [69]:
cluster_centers = km_cluster.cluster_centers_
print('cluster_centers shape : ', cluster_centers.shape)
print(cluster_centers)

cluster_centers shape :  (3, 4611)
[[0.01005322 0.         0.         ... 0.00706287 0.         0.        ]
 [0.         0.00092551 0.         ... 0.         0.         0.        ]
 [0.         0.00099499 0.00174637 ... 0.         0.00183397 0.00144581]]


### 군집별 top n 핵심단어, 대상 파일명 등

In [72]:
feature_names = tfidf_vect.get_feature_names()
# print(feature_names)
cluster_details = {}

# cluster_centers를 정렬 - 정렬 결과의 index값을 얻기 위해서 argsort 실행(내림차순)
centroid_feature_ordered_ind = km_cluster.cluster_centers_.argsort()[:, ::-1]

In [75]:
# 개별 군집별로 iteration하면서 핵심단어, 그 단어의 중심 위치 상대값, 대상 파일명
# cluster_details에 입력
for cluster_num in range(3) :
    # 개별 군집별 정보를 담을 데이터 초기화
    cluster_details[cluster_num] = {}
    cluster_details[cluster_num]['cluster'] = cluster_num
    # cluster_centers_.argsort()[:, ::-1]로 얻은 index를 이용하여 top n
    # 피처 단어들의 인덱스, 각 클러스터 별 10개의 인덱스
    top_feature_indexes = centroid_feature_ordered_ind[cluster_num, :10]
    # 단어 추출
    top_features = [feature_names[ind] for ind in top_feature_indexes]
    # 단어들의 상대적 거리 추출
    top_feature_values = km_cluster.cluster_centers_[cluster_num, top_feature_indexes].tolist()
    
    # cluster_details 딕셔너리 객체에 개별 군집별 핵심 단어와 중심위치 상대값,
    # 그리고 해당 파일명 저장
    cluster_details[cluster_num]['top_features'] = top_features
    cluster_details[cluster_num]['top_features_value'] = top_feature_values
    filenames = document_df[document_df['cluster_label']==cluster_num]['filename']
    filenames = filenames.values.tolist()
    cluster_details[cluster_num]['filenames'] = filenames
    
# 완성된 자료 출력
for cluster_num, cluster_detail in cluster_details.items() :
    print('####### Cluster {0}'.format(cluster_num))
    print('Top features: ', cluster_detail['top_features'])
    print('Reviews 파일명 : ', cluster_detail['filenames'][:7])
    print('='*50)

####### Cluster 0
Top features:  ['screen', 'battery', 'keyboard', 'battery life', 'life', 'kindle', 'direction', 'video', 'size', 'voice']
Reviews 파일명 :  ['accuracy_garmin_nuvi_255W_gps', 'battery-life_amazon_kindle', 'battery-life_ipod_nano_8gb', 'battery-life_netbook_1005ha', 'buttons_amazon_kindle', 'directions_garmin_nuvi_255W_gps', 'display_garmin_nuvi_255W_gps']
####### Cluster 1
Top features:  ['interior', 'seat', 'mileage', 'comfortable', 'gas', 'gas mileage', 'transmission', 'car', 'performance', 'quality']
Reviews 파일명 :  ['comfort_honda_accord_2008', 'comfort_toyota_camry_2007', 'gas_mileage_toyota_camry_2007', 'interior_honda_accord_2008', 'interior_toyota_camry_2007', 'mileage_honda_accord_2008', 'performance_honda_accord_2008']
####### Cluster 2
Top features:  ['room', 'hotel', 'service', 'staff', 'food', 'location', 'bathroom', 'clean', 'price', 'parking']
Reviews 파일명 :  ['bathroom_bestwestern_hotel_sfo', 'food_holiday_inn_london', 'food_swissotel_chicago', 'free_bestwes